In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
# Define constants
IMG_WIDTH, IMG_HEIGHT = 256, 256
BATCH_SIZE = 32

# Load VGG16 model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(IMG_WIDTH, IMG_HEIGHT, 3))

# Freeze base model layers
for layer in base_model.layers:
    layer.trainable = False

# Add new layers for classification
x = base_model.output
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(6, activation='softmax')(x)

# Create new model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Load the custom dataset
train_dir = '/content/drive/MyDrive/Eugenia & Thasniya - Term paper/New Plant Diseases Dataset(Augmented)/train'

# Train the model
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(train_dir, target_size=(IMG_WIDTH, IMG_HEIGHT), batch_size=BATCH_SIZE, class_mode='categorical')
history = model.fit(train_generator, epochs=3)

# Define a function to predict the class of a single image
def predict_image(image_path):
    # Load the image
    img = load_img(image_path, target_size=(IMG_WIDTH, IMG_HEIGHT))

    # Convert the image to an array
    img_array = img_to_array(img)

    # Rescale the image
    img_array = img_array / 255.0

    # Expand the dimensions of the array
    img_array = np.expand_dims(img_array, axis=0)

    # Predict the class of the image
    predictions = model.predict(img_array)

    # Get the class label with the highest probability
    class_label = np.argmax(predictions[0])

    # Get the class labels
    class_labels = os.listdir(train_dir)

    # Print the predicted class label
    print("Predicted class: ", class_labels[class_label])


Found 300 images belonging to 6 classes.
Epoch 1/3
10/10 ━━━━━━━━━━━━━━━━━━━━ 214s 20s/step - accuracy: 0.2009 - loss: 4.4916
Epoch 2/3
10/10 ━━━━━━━━━━━━━━━━━━━━ 258s 21s/step - accuracy: 0.4975 - loss: 1.3600
Epoch 3/3
10/10 ━━━━━━━━━━━━━━━━━━━━ 262s 21s/step - accuracy: 0.8010 - loss: 0.5955


In [6]:
image_path = '/content/drive/MyDrive/Eugenia & Thasniya - Term paper/New Plant Diseases Dataset(Augmented)/train/Tomato___healthy/Copy of d3997d9d-f607-44f2-9473-b41db6b1116b___GH_HL Leaf 473.JPG'
predict_image(image_path)

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Predicted class:  Tomato___healthy
